In [1]:
import psycopg2
import pandas as pd
import requests
from io import StringIO

In [ ]:
query = '''
    SELECT
      date,
      daily_revenue,
      revenue_growth_abs,
      COALESCE(ROUND(revenue_growth_abs / lag::DECIMAL * 100, 1), 0) AS revenue_growth_percentage
    FROM (
      SELECT
        date,
        daily_revenue,
        COALESCE(
          daily_revenue - LAG(daily_revenue) OVER(ORDER BY date),
          0
        ) AS revenue_growth_abs,
        LAG(daily_revenue) OVER(ORDER BY date) AS lag
      FROM (
        SELECT
          creation_time::DATE AS date,
          SUM(price) AS daily_revenue
        FROM (
          SELECT
            *
          FROM (
            SELECT
              *,
              unnest(product_ids) AS product_id
            FROM
              orders
          ) t1
          JOIN products USING(product_id)
        ) t2
        WHERE
          order_id NOT IN (
            SELECT
              order_id
            FROM
              user_actions
            WHERE
              action = 'cancel_order'
          )
        GROUP BY
          creation_time::DATE
      ) t3
    ) t4
    '''

In [9]:
# URL для выполнения запроса
redash_url = 'http://redash.public.karpov.courses/api/queries/51119/results.csv?api_key=nN4z8WpasuiJGBJnWfXGJl4r4BTuAfRh0hb0eYPZ'

# Выполнение запроса и получение данных в формате CSV
response = requests.get(redash_url)

# Проверка успешности выполнения запроса
if response.status_code == 200:
    # Преобразование данных в DataFrame
    df = pd.read_csv(StringIO(response.text))
    
else:
    print(f"Error: {response.status_code}")
    print(response.text)

In [10]:
df

,date,daily_revenue,revenue_growth_abs,revenue_growth_percentage
0,24/08/22,49924,0,0.0
1,25/08/22,430860,380936,763.0
2,26/08/22,534766,103906,24.1
3,27/08/22,817053,282287,52.8
4,28/08/22,1133370,316317,38.7
5,29/08/22,1279891,146521,12.9
6,30/08/22,1279377,-514,0.0
7,31/08/22,1312720,33343,2.6
8,01/09/22,1406101,93381,7.1
9,02/09/22,1907107,501006,35.6


**Вывод:** На основе информации в таблицах orders и products рассчитал ежедневную выручку сервиса и отразил её в колонке daily_revenue. Затем с помощью оконных функций и функций смещения посчитал ежедневный прирост выручки. Прирост выручки отразил как в абсолютных значениях, так и в % относительно предыдущего дня.